In [1]:
from fr_models.config import *

fr_model_names = list(threshold_lfw.keys())
# fr_model_names = ['testirse50', 'testir152', 'testmobileface', 'testfacenet']
print("---------------------all fr models----------------------")
fr_model_names

---------------------all fr models----------------------


['ShuffleNet_V2_GDConv-stride1',
 'Mobilenet',
 'MobilenetV2-stride1',
 'CosFace',
 'FaceNet-VGGFace2',
 'Mobilenet-stride1',
 'SphereFace',
 'MobileFace',
 'FaceNet-casia',
 'MobilenetV2',
 'ArcFace',
 'ResNet50',
 'ShuffleNet_V1_GDConv',
 'ResNet50-casia',
 'IR50-Am',
 'IR50-ArcFace',
 'IR50-CosFace',
 'IR50-Softmax',
 'IR50-SphereFace',
 'IR50-PGDSoftmax',
 'IR50-TradesSoftmax',
 'IR50-PGDArcFace',
 'IR50-TradesArcFace',
 'IR50-PGDCosFace',
 'IR50-TradesCosFace',
 'IR50-PGDAm',
 'IR50-PGDSphereFace',
 'CASIA-Am',
 'CASIA-ArcFace',
 'CASIA-CosFace',
 'CASIA-Softmax',
 'CASIA-SphereFace',
 'IR50-Softmax-BR',
 'IR50-Softmax-JPEG',
 'IR50-Softmax-RP',
 'testir152',
 'testirse50',
 'testmobileface',
 'testfacenet']

In [2]:
import torch
import torchvision
import torch.nn.functional as F
from utils.dataset import Dataset
from fr_models.get_model import getmodel
from tqdm import tqdm

def loadDataset(dataset_dir, mean, std, mode, batch_size):
    dataset = Dataset(dataset_dir, mode)
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=mean, std=std), 
    ])
    dataset.start_batch_queue(
        batch_size=batch_size,
        batch_format="random_samples",
        transforms=transforms,
        num_threads=2,
        untarget_same=False
    )
    return dataset

def evalFRModel(model, img_shape, mode, dataset):
    epoch_size = 6000 // batch_size
    simi_scores = []
    for _ in tqdm(range(0, epoch_size), total=epoch_size):
        batch = dataset.pop_batch_queue()
        with torch.no_grad():
            source_faces = batch['sources'].cuda()
            target_faces = batch['targets'].cuda()
            emb_source_faces = model.forward(F.interpolate((source_faces*0.5+0.5)*255, size=img_shape, mode='bilinear'))
            emb_target_faces = model.forward(F.interpolate((target_faces*0.5+0.5)*255, size=img_shape, mode='bilinear'))
            batch_scores = torch.cosine_similarity(emb_source_faces, emb_target_faces)
            simi_scores.extend(batch_scores.tolist())
    th = threshold_lfw[model_name]['cos']
    print("识别阈值:", th)
    mean_simi = sum(simi_scores)/len(simi_scores)
    print("平均相似分数:", mean_simi)
    success_rate = sum(score > th if mode=="untarget" else score < th for score in simi_scores) / len(simi_scores)
    print(model_name, "benchmark rate:%f" % threshold_lfw[model_name]['cos_acc'])
    print(model_name, "success rate:%f" % success_rate)
    return th, mean_simi, threshold_lfw[model_name]['cos_acc'], success_rate

In [4]:
from fr_models.get_model import getmodel
import json

batch_size=32
# your dataset's directory
dataset_dir="C:/yy/datasets/lfw/lfw-aligned-112x112"
untarget_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="untarget",
                                   batch_size=batch_size)
target_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="target",
                                   batch_size=batch_size)
benchmark = {}
print("use dataset %s:" % dataset_dir)
for model_name in fr_model_names:
    with torch.no_grad():
        benchmark[model_name]={}
        print("evaluation %s now:" % (model_name))
        model, img_shape = getmodel(model_name)
        print("    TAR:")
        benchmark[model_name]['TAR']=evalFRModel(model, img_shape, "untarget", untarget_dataset)
        print("    FAR:")
        benchmark[model_name]['FAR']=evalFRModel(model, img_shape, "target", target_dataset)

print(json.dumps(benchmark, indent=2))

9164 images of 1680 classes loaded
13233 images of 5749 classes loaded
use dataset C:/yy/datasets/lfw/lfw-aligned-112x112:
evaluation ShuffleNet_V2_GDConv-stride1 now:
Load existing checkpoint
    TAR:


  0%|          | 0/187 [00:00<?, ?it/s]C:\yy\installed_software\Anaconda3\envs\AdvFaceGAN\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

识别阈值: 0.22403636574745178
平均相似分数: 0.5562467525761704
ShuffleNet_V2_GDConv-stride1 benchmark rate:0.991500
ShuffleNet_V2_GDConv-stride1 success rate:0.983456
    FAR:


100%|██████████| 187/187 [00:12<00:00, 14.48it/s]


识别阈值: 0.22403636574745178
平均相似分数: 0.034676948938980066
ShuffleNet_V2_GDConv-stride1 benchmark rate:0.991500
ShuffleNet_V2_GDConv-stride1 success rate:0.994987
evaluation Mobilenet now:
Load existing checkpoint
    TAR:


...

识别阈值: 0.301611
平均相似分数: 0.01911578172311077
testmobileface benchmark rate:0.996992
testmobileface success rate:0.995989
evaluation testfacenet now:
    TAR:


100%|██████████| 187/187 [00:21<00:00,  8.51it/s]


识别阈值: 0.409131
平均相似分数: 0.7569139939913386
testfacenet benchmark rate:0.987634
testfacenet success rate:0.991979
    FAR:


100%|██████████| 187/187 [00:21<00:00,  8.67it/s]

识别阈值: 0.409131
平均相似分数: 0.03188121795575234
testfacenet benchmark rate:0.987634
testfacenet success rate:0.988636
{
  "ShuffleNet_V2_GDConv-stride1": {
    "TAR": [
      0.22403636574745178,
      0.5562467525761704,
      0.9915,
      0.9834558823529411
    ],
    "FAR": [
      0.22403636574745178,
      0.034676948938980066,
      0.9915,
      0.9949866310160428
    ]
  },
  "Mobilenet": {
    "TAR": [
      0.15846577286720276,
      0.5287337219143377,
      0.9935,
      0.9899732620320856
    ],
    "FAR": [
      0.15846577286720276,
      0.00530493123069926,
      0.9935,
      0.9956550802139037
    ]
  },
  "MobilenetV2-stride1": {
    "TAR": [
      0.24340271949768066,
      0.572384514267255,
      0.9905,
      0.9869652406417112
    ],
    "FAR": [
      0.24340271949768066,
      0.02833695003155396,
      0.9905,
      0.9941510695187166
    ]
  },
  "CosFace": {
    "TAR": [
      0.24625787138938904,
      0.586980783589742,
      0.9868333333333333,
      0.9817

In [3]:
from fr_models.get_model import getmodel
import json

batch_size=32
dataset_dir="C:/yy/datasets/CASIA-WebFace/casia-aligned-112x112"
untarget_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="untarget",
                                   batch_size=batch_size)
target_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="target",
                                   batch_size=batch_size)
benchmark = {}
print("use dataset %s:" % dataset_dir)
for model_name in fr_model_names:
    with torch.no_grad():
        benchmark[model_name]={}
        print("evaluation %s now:" % (model_name))
        model, img_shape = getmodel(model_name)
        print("    TAR:")
        benchmark[model_name]['TAR']=evalFRModel(model, img_shape, "untarget", untarget_dataset)
        print("    FAR:")
        benchmark[model_name]['FAR']=evalFRModel(model, img_shape, "target", target_dataset)

print(json.dumps(benchmark, indent=2))

455594 images of 10575 classes loaded
455594 images of 10575 classes loaded
use dataset C:/yy/datasets/CASIA-WebFace/casia-aligned-112x112:
evaluation ShuffleNet_V2_GDConv-stride1 now:
Load existing checkpoint
    TAR:


  0%|          | 0/187 [00:00<?, ?it/s]C:\yy\installed_software\Anaconda3\envs\AdvFaceGAN\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

识别阈值: 0.22403636574745178
平均相似分数: 0.38271826634994077
ShuffleNet_V2_GDConv-stride1 benchmark rate:0.991500
ShuffleNet_V2_GDConv-stride1 success rate:0.801972
    FAR:


100%|██████████| 187/187 [00:10<00:00, 17.17it/s]


识别阈值: 0.22403636574745178
平均相似分数: 0.035588228471753344
ShuffleNet_V2_GDConv-stride1 benchmark rate:0.991500
ShuffleNet_V2_GDConv-stride1 success rate:0.992981
evaluation Mobilenet now:
Load existing checkpoint
    TAR:


...

识别阈值: 0.301611
平均相似分数: 0.02459002282872938
testmobileface benchmark rate:0.996992
testmobileface success rate:0.997326
evaluation testfacenet now:
    TAR:


100%|██████████| 187/187 [00:19<00:00,  9.40it/s]


识别阈值: 0.409131
平均相似分数: 0.5783906054898118
testfacenet benchmark rate:0.987634
testfacenet success rate:0.818349
    FAR:


100%|██████████| 187/187 [00:44<00:00,  4.19it/s]

识别阈值: 0.409131
平均相似分数: 0.015959535572580733
testfacenet benchmark rate:0.987634
testfacenet success rate:0.992981
{
  "ShuffleNet_V2_GDConv-stride1": {
    "TAR": [
      0.22403636574745178,
      0.38271826634994077,
      0.9915,
      0.8019719251336899
    ],
    "FAR": [
      0.22403636574745178,
      0.035588228471753344,
      0.9915,
      0.99298128342246
    ]
  },
  "Mobilenet": {
    "TAR": [
      0.15846577286720276,
      0.3427625058964642,
      0.9935,
      0.8153409090909091
    ],
    "FAR": [
      0.15846577286720276,
      0.008124033538187308,
      0.9935,
      0.9939839572192514
    ]
  },
  "MobilenetV2-stride1": {
    "TAR": [
      0.24340271949768066,
      0.40572042708858047,
      0.9905,
      0.7916109625668449
    ],
    "FAR": [
      0.24340271949768066,
      0.03876266579840483,
      0.9905,
      0.990975935828877
    ]
  },
  "CosFace": {
    "TAR": [
      0.24625787138938904,
      0.4719177234896401,
      0.9868333333333333,
      0.8